In [1]:
import numpy as np
from numpy import float32,int32
np.random.seed(42)
import tensorflow as tf
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
import seaborn as sns
from numpy import mean
from numpy import std
from numpy import dstack
from keras import optimizers
tf.set_random_seed(42)
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout,Permute,Reshape
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM, Dense, Embedding, Flatten, RepeatVector, TimeDistributed, Concatenate
from keras.callbacks import TensorBoard, ModelCheckpoint
import os
import matplotlib
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
%matplotlib inline
matplotlib.style.use('ggplot')
DATADIR = 'HARDataset'

SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}


''' A confusion matrix is a matrix (table) that can be used to measure the performance of an machine learning algorithm, 
    usually a supervised learning one. Each row of the confusion matrix represents the instances of an actual class and 
    each column represents the instances of a predicted class.
    argmax is a function which gives the index of the greatest number in the given row or column and the row or column 
    can be decided using axis attribute of argmax funcion. 
    If we give axis=0 then it will give the index from columns and if we give axis=1 then it will give the index from rows.
    If A[1:, 2] it will first fetch the values from 1st row on wards and the only 2nd column value from 
    those rows, then it will find the index of max value from into the resulted matrix.
    pandas.crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, 
    margins_name='All', dropna=True, normalize=False) 
    Compute a simple cross tabulation of two (or more) factors. By default computes a frequency table of the factors unless 
    an array of values and an aggregation function are passed.''' 
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])
    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

# Utility function to read the data from csv file                     
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []
    for signal in SIGNALS:
        filename = '{0}/{1}/Inertial Signals/{2}_{3}.txt'.format(DATADIR,subset,signal,subset)
        signals_data.append(
            _read_csv(filename).as_matrix()
        )
    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    '''>>> x = np.ones((1, 2, 3))
       >>> np.transpose(x, (1, 0, 2)).shape
       (2, 1, 3)'''
    return np.transpose(signals_data, (1, 2, 0))

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = '{0}/{1}/y_{2}.txt'.format(DATADIR,subset,subset)
    y = _read_csv(filename)[0]
    return pd.get_dummies(y).as_matrix()
"""
>>> s = pd.Series(list('abca'))
>>> pd.get_dummies(s)
   a  b  c
0  1  0  0
1  0  1  0
2  0  0  1
3  1  0  0"""

def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
 shape:   
[samples, time steps, features]
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')
    print(X_train.shape)
    return X_train, X_test, y_train, y_test

LABELS = ['WALKING','WALKING_UPSTAIRS','WALKING_DOWNSTAIRS','SITTING','STANDING','LAYING']
lables=np.array(LABELS)

def one_hot(y_):
    """
    Function to encode output labels from number indexes.
    E.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    """
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

Using TensorFlow backend.


In [31]:
# run an experiment
X_train, X_test, Y_train, Y_test = load_data()
y_test=Y_test.argmax(1)
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)
tick_marks = np.arange(n_classes)
# fit and evaluate a model
verbose, epochs, batch_size, validation_split = 1, 30, 32, 0.2
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(256)))
#model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))        
#what happen here?
#model.add(Bidirectional(LSTM(100, input_shape=(n_timesteps,n_features))))
#model.add(Bidirectional(LSTM(128, input_shape=(100,6))))
#what happen here?
#model.add(Dropout(0.5))
model.add(Dense(156, activation='relu'))
#model.add(Dense(n_outputs, activation='softmax'))
model.add(Dense(6, activation='softmax'))
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# fit network
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,validation_split=validation_split,shuffle=False)

C:\Users\User\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:84: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\User\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:103: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


(7352, 128, 9)
Train on 5881 samples, validate on 1471 samples
Epoch 1/30
5881/5881 [==============================] - 327s 56ms/step - loss: 1.3481 - acc: 0.4064 - val_loss: 1.8786 - val_acc: 0.3474
Epoch 2/30
5881/5881 [==============================] - 309s 53ms/step - loss: 0.8884 - acc: 0.5608 - val_loss: 0.8021 - val_acc: 0.6805
Epoch 3/30
5881/5881 [==============================] - 327s 56ms/step - loss: 0.7567 - acc: 0.6742 - val_loss: 0.7189 - val_acc: 0.7573
Epoch 4/30
5881/5881 [==============================] - 253s 43ms/step - loss: 0.6351 - acc: 0.7213 - val_loss: 0.6792 - val_acc: 0.7492
Epoch 5/30
5881/5881 [==============================] - 168s 29ms/step - loss: 0.5347 - acc: 0.7662 - val_loss: 0.6023 - val_acc: 0.8321
Epoch 6/30
5881/5881 [==============================] - 344s 58ms/step - loss: 0.4881 - acc: 0.8080 - val_loss: 0.7144 - val_acc: 0.7974
Epoch 7/30
5881/5881 [==============================] - 346s 59ms/step - loss: 0.4362 - acc: 0.8269 - val_loss: 0.6

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 126, 32)           896       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 124, 32)           3104      
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 122, 32)           3104      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 61, 32)            0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 512)               591872    
_________________________________________________________________
dense_11 (Dense)             (None, 156)               80028     
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 942       
Total para

In [33]:
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\User\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:84: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


(7352, 128, 9)


C:\Users\User\Miniconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:103: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [34]:
X_train.shape

(7352, 128, 9)

In [35]:
X_test.shape

(2947, 128, 9)

In [36]:
X_train[0]

array([[ 1.808515e-04,  1.076681e-02,  5.556068e-02, ...,  1.012817e+00,
        -1.232167e-01,  1.029341e-01],
       [ 1.013856e-02,  6.579480e-03,  5.512483e-02, ...,  1.022833e+00,
        -1.268756e-01,  1.056872e-01],
       [ 9.275574e-03,  8.928878e-03,  4.840473e-02, ...,  1.022028e+00,
        -1.240037e-01,  1.021025e-01],
       ...,
       [-1.147484e-03,  1.714439e-04,  2.647864e-03, ...,  1.018445e+00,
        -1.240696e-01,  1.003852e-01],
       [-2.222655e-04,  1.574181e-03,  2.381057e-03, ...,  1.019372e+00,
        -1.227451e-01,  9.987355e-02],
       [ 1.575500e-03,  3.070189e-03, -2.269757e-03, ...,  1.021171e+00,
        -1.213260e-01,  9.498741e-02]])

In [37]:
Y_train[0]

array([0, 0, 0, 0, 1, 0], dtype=uint8)

In [38]:
predict=model.predict(X_test)
#print("predict#################",predict)
pred_index_total=[]
for pred in predict:
    pred_index = []
    pred_list=pred.tolist()
    index_max=pred_list.index(max(pred_list))
    pred_index.append(index_max)
    pred_index_total.append(np.array(pred_index))
_, accuracy = model.evaluate(X_test, Y_test, batch_size=200, verbose=1)
print(accuracy)

2947/2947 [==============================] - 6s 2ms/step
0.9097387225174038


In [39]:
# remove the 1st output layer Dense with 6 nodes
model1 = Model(inputs=model.inputs, outputs=model.layers[-2].output)
model2 = Model(inputs=model.inputs, outputs=model.layers[-3].output)
dense_output_train= model1.predict(X_train)
print('dense_output_train: ',dense_output_train)
dense_output_test = model1.predict(X_test)
print('dense_output_test: ',dense_output_test)
lstm_output_train= model2.predict(X_train)
print('lstm_output_train: ',lstm_output_train)
lstm_output_test = model2.predict(X_test)
print('lstm_output_test: ',lstm_output_test)

dense_output_train:  [[0.         0.39142996 0.         ... 1.7438701  0.         0.        ]
 [0.         0.3625081  0.         ... 1.701524   0.         0.        ]
 [0.         0.3957115  0.         ... 1.7456783  0.         0.        ]
 ...
 [0.         0.04011294 0.         ... 0.2062599  0.         1.6150682 ]
 [0.         0.         0.         ... 0.         0.         1.7081707 ]
 [0.         0.01882551 0.         ... 0.         0.         1.5202266 ]]
dense_output_test:  [[0.         0.46035108 0.         ... 1.9470792  0.         0.        ]
 [0.         0.46077326 0.         ... 2.024555   0.         0.        ]
 [0.         0.45782265 0.         ... 2.0439262  0.         0.        ]
 ...
 [0.         0.08639769 0.         ... 0.04566081 0.         1.4552189 ]
 [0.         0.14537519 0.         ... 0.         0.08236421 2.1482308 ]
 [0.         0.         0.         ... 0.         0.         2.0133736 ]]
lstm_output_train:  [[ 7.47666478e-01  9.63047594e-02  5.48575640e-01 .

In [40]:
np.savetxt('uci_dense_output_train.csv',dense_output_train, delimiter=',', fmt='%.7e')
np.savetxt('uci_dense_output_test.csv',dense_output_test, delimiter=',', fmt='%.7e')
np.savetxt('uci_lstm_output_train.csv',lstm_output_train, delimiter=',', fmt='%.7e')
np.savetxt('uci_lstm_output_test.csv',lstm_output_test, delimiter=',', fmt='%.7e')

In [41]:
output_train=model.predict(X_train)
output_test=model.predict(X_test)

In [42]:
output_test.shape

(2947, 6)

In [43]:
np.savetxt('uci_probability_output_train2.csv',output_train, delimiter=',',fmt='%.7e')
np.savetxt('uci_probability_output_test2.csv',output_test, delimiter=',',fmt='%.7e')

In [44]:
train_output=np.argmax(output_train,axis=1)
np.savetxt('uci_probability_output_train.csv',train_output, delimiter=',',fmt='%i')
test_output=np.argmax(output_test,axis=1)
np.savetxt('uci_probability_output_test.csv',test_output, delimiter=',',fmt='%i')

In [45]:
train_output

array([4, 4, 4, ..., 1, 1, 1], dtype=int64)

In [46]:
test_output

array([4, 4, 4, ..., 1, 1, 1], dtype=int64)

In [47]:
result_train=[]
for i in train_output:
    if i == 0:
        result_train.append('WALKING')
    if i == 1:
        result_train.append('WALKING_UPSTAIRS')
    if i == 2:
        result_train.append('WALKING_DOWNSTAIRS')
    if i == 3:
        result_train.append('SITTING')
    if i == 4:
        result_train.append('STANDING')
    if i == 5:
        result_train.append('LAYING')

print(result_train)
#print [s.replace('HD\,' , 'HD ') for s in L]
#np.char.replace(classification,('1','2','3','4','5','6'), ('WALKING','WALKING_UPSTAIRS','WALKING_DOWNSTAIRS','SITTING','STANDING','LAYING'))

['STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING'

In [48]:
result_test=[]
for i in test_output:
    if i == 0:
        result_test.append('WALKING')
    if i == 1:
        result_test.append('WALKING_UPSTAIRS')
    if i == 2:
        result_test.append('WALKING_DOWNSTAIRS')
    if i == 3:
        result_test.append('SITTING')
    if i == 4:
        result_test.append('STANDING')
    if i == 5:
        result_test.append('LAYING')

print(result_test)

['STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 

In [49]:
npa_train=np.asarray(result_train)
npa_test=np.asarray(result_test)

In [50]:
npa_test.shape

(2947,)

In [51]:
np.savetxt('uci_probability_output_train_classfication.csv',npa_train, delimiter=',',fmt='%s')
np.savetxt('uci_probability_output_test_classfication.csv',npa_test, delimiter=',',fmt='%s')

In [52]:
df_dense_test=pd.DataFrame(dense_output_test)
df_dense_train=pd.DataFrame(dense_output_train)
df_lstm_test=pd.DataFrame(lstm_output_test)
df_lstm_train=pd.DataFrame(lstm_output_train)

df_train=pd.DataFrame(npa_train)
df_test=pd.DataFrame(npa_test)

from sklearn.preprocessing import normalize
normalized_df_dense_test=pd.DataFrame(normalize(df_dense_test))
normalized_df_dense_train=pd.DataFrame(normalize(df_dense_train))
normalized_df_lstm_test=pd.DataFrame(normalize(df_lstm_test))
normalized_df_lstm_train=pd.DataFrame(normalize(df_lstm_train))

In [53]:
merged_dense_test_file=pd.concat([normalized_df_dense_test,df_test],axis=1)
merged_dense_train_file=pd.concat([normalized_df_dense_train,df_train],axis=1)
merged_lstm_test_file=pd.concat([normalized_df_lstm_test,df_test],axis=1)
merged_lstm_train_file=pd.concat([normalized_df_lstm_train,df_train],axis=1)

In [54]:
np.savetxt('uci_merged_dense_test_file.csv',merged_dense_test_file, delimiter=',',fmt='%s')
np.savetxt('uci_merged_dense_train_file.csv',merged_dense_train_file, delimiter=',',fmt='%s')
np.savetxt('uci_merged_lstm_test_file.csv',merged_lstm_test_file, delimiter=',',fmt='%s')
np.savetxt('uci_merged_lstm_train_file.csv',merged_lstm_train_file, delimiter=',',fmt='%s')

In [55]:
merged_dense_test_file

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,0
0,0.0,0.034762,0.0,0.0,0.0,0.234697,0.000000,0.0,0.172232,0.203156,...,0.0,0.000000,0.0,0.070662,0.000000,0.0,0.147027,0.000000,0.000000,STANDING
1,0.0,0.034053,0.0,0.0,0.0,0.232860,0.000000,0.0,0.173502,0.203003,...,0.0,0.000000,0.0,0.071925,0.000000,0.0,0.149622,0.000000,0.000000,STANDING
2,0.0,0.033420,0.0,0.0,0.0,0.235055,0.000000,0.0,0.175799,0.199896,...,0.0,0.000000,0.0,0.067573,0.000000,0.0,0.149203,0.000000,0.000000,STANDING
3,0.0,0.034116,0.0,0.0,0.0,0.233866,0.000000,0.0,0.175386,0.201393,...,0.0,0.000000,0.0,0.068754,0.000000,0.0,0.149588,0.000000,0.000000,STANDING
4,0.0,0.033806,0.0,0.0,0.0,0.234499,0.000000,0.0,0.175545,0.201011,...,0.0,0.000000,0.0,0.068596,0.000000,0.0,0.149658,0.000000,0.000000,STANDING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.0,0.000000,0.0,0.0,0.0,0.270403,0.029007,0.0,0.271175,0.000000,...,0.0,0.033557,0.0,0.000000,0.048693,0.0,0.000000,0.012626,0.133427,WALKING_UPSTAIRS
2943,0.0,0.000000,0.0,0.0,0.0,0.237460,0.063615,0.0,0.285803,0.000000,...,0.0,0.021019,0.0,0.000000,0.049077,0.0,0.000000,0.000000,0.182375,WALKING_UPSTAIRS
2944,0.0,0.006900,0.0,0.0,0.0,0.283303,0.000000,0.0,0.283703,0.000000,...,0.0,0.000000,0.0,0.000000,0.014266,0.0,0.003647,0.000000,0.116224,WALKING_UPSTAIRS
2945,0.0,0.010210,0.0,0.0,0.0,0.227192,0.049463,0.0,0.285762,0.000000,...,0.0,0.089696,0.0,0.000000,0.035702,0.0,0.000000,0.005784,0.150867,WALKING_UPSTAIRS


In [68]:
Y_train

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

In [66]:
y_train_hot=np.argmax(Y_train,axis=1)
y_train_hot
y_test_hot=np.argmax(Y_test,axis=1)
y_test_hot

array([4, 4, 4, ..., 1, 1, 1], dtype=int64)

In [67]:
result_train2=[]
for i in y_train_hot:
    if i == 0:
        result_train2.append('WALKING')
    if i == 1:
        result_train2.append('WALKING_UPSTAIRS')
    if i == 2:
        result_train2.append('WALKING_DOWNSTAIRS')
    if i == 3:
        result_train2.append('SITTING')
    if i == 4:
        result_train2.append('STANDING')
    if i == 5:
        result_train2.append('LAYING')

print(result_train)

['STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING'

In [69]:
result_test2=[]
for i in y_test_hot:
    if i == 0:
        result_test2.append('WALKING')
    if i == 1:
        result_test2.append('WALKING_UPSTAIRS')
    if i == 2:
        result_test2.append('WALKING_DOWNSTAIRS')
    if i == 3:
        result_test2.append('SITTING')
    if i == 4:
        result_test2.append('STANDING')
    if i == 5:
        result_test2.append('LAYING')
print(result_test2)

['STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'STANDING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'SITTING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'LAYING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'WALKING', 'W

In [70]:
df_train2=pd.DataFrame(result_train2)
df_test2=pd.DataFrame(result_test2)

In [71]:
normalized_df_dense_test

,0,1,2,3,4,5,6,7,8,9,...,146,147,148,149,150,151,152,153,154,155
0,0.0,0.034762,0.0,0.0,0.0,0.234697,0.000000,0.0,0.172232,0.203156,...,0.000000,0.0,0.000000,0.0,0.070662,0.000000,0.0,0.147027,0.000000,0.000000
1,0.0,0.034053,0.0,0.0,0.0,0.232860,0.000000,0.0,0.173502,0.203003,...,0.000000,0.0,0.000000,0.0,0.071925,0.000000,0.0,0.149622,0.000000,0.000000
2,0.0,0.033420,0.0,0.0,0.0,0.235055,0.000000,0.0,0.175799,0.199896,...,0.000000,0.0,0.000000,0.0,0.067573,0.000000,0.0,0.149203,0.000000,0.000000
3,0.0,0.034116,0.0,0.0,0.0,0.233866,0.000000,0.0,0.175386,0.201393,...,0.000000,0.0,0.000000,0.0,0.068754,0.000000,0.0,0.149588,0.000000,0.000000
4,0.0,0.033806,0.0,0.0,0.0,0.234499,0.000000,0.0,0.175545,0.201011,...,0.000000,0.0,0.000000,0.0,0.068596,0.000000,0.0,0.149658,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.0,0.000000,0.0,0.0,0.0,0.270403,0.029007,0.0,0.271175,0.000000,...,0.110900,0.0,0.033557,0.0,0.000000,0.048693,0.0,0.000000,0.012626,0.133427
2943,0.0,0.000000,0.0,0.0,0.0,0.237460,0.063615,0.0,0.285803,0.000000,...,0.116924,0.0,0.021019,0.0,0.000000,0.049077,0.0,0.000000,0.000000,0.182375
2944,0.0,0.006900,0.0,0.0,0.0,0.283303,0.000000,0.0,0.283703,0.000000,...,0.080091,0.0,0.000000,0.0,0.000000,0.014266,0.0,0.003647,0.000000,0.116224
2945,0.0,0.010210,0.0,0.0,0.0,0.227192,0.049463,0.0,0.285762,0.000000,...,0.101264,0.0,0.089696,0.0,0.000000,0.035702,0.0,0.000000,0.005784,0.150867


In [72]:
merged_dense_test_file2=pd.concat([normalized_df_dense_test,df_test2],axis=1)
merged_dense_train_file2=pd.concat([normalized_df_dense_train,df_train2],axis=1)
merged_lstm_test_file2=pd.concat([normalized_df_lstm_test,df_test2],axis=1)
merged_lstm_train_file2=pd.concat([normalized_df_lstm_train,df_train2],axis=1)

In [73]:
merged_dense_test_file2

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,0
0,0.0,0.034762,0.0,0.0,0.0,0.234697,0.000000,0.0,0.172232,0.203156,...,0.0,0.000000,0.0,0.070662,0.000000,0.0,0.147027,0.000000,0.000000,STANDING
1,0.0,0.034053,0.0,0.0,0.0,0.232860,0.000000,0.0,0.173502,0.203003,...,0.0,0.000000,0.0,0.071925,0.000000,0.0,0.149622,0.000000,0.000000,STANDING
2,0.0,0.033420,0.0,0.0,0.0,0.235055,0.000000,0.0,0.175799,0.199896,...,0.0,0.000000,0.0,0.067573,0.000000,0.0,0.149203,0.000000,0.000000,STANDING
3,0.0,0.034116,0.0,0.0,0.0,0.233866,0.000000,0.0,0.175386,0.201393,...,0.0,0.000000,0.0,0.068754,0.000000,0.0,0.149588,0.000000,0.000000,STANDING
4,0.0,0.033806,0.0,0.0,0.0,0.234499,0.000000,0.0,0.175545,0.201011,...,0.0,0.000000,0.0,0.068596,0.000000,0.0,0.149658,0.000000,0.000000,STANDING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.0,0.000000,0.0,0.0,0.0,0.270403,0.029007,0.0,0.271175,0.000000,...,0.0,0.033557,0.0,0.000000,0.048693,0.0,0.000000,0.012626,0.133427,WALKING_UPSTAIRS
2943,0.0,0.000000,0.0,0.0,0.0,0.237460,0.063615,0.0,0.285803,0.000000,...,0.0,0.021019,0.0,0.000000,0.049077,0.0,0.000000,0.000000,0.182375,WALKING_UPSTAIRS
2944,0.0,0.006900,0.0,0.0,0.0,0.283303,0.000000,0.0,0.283703,0.000000,...,0.0,0.000000,0.0,0.000000,0.014266,0.0,0.003647,0.000000,0.116224,WALKING_UPSTAIRS
2945,0.0,0.010210,0.0,0.0,0.0,0.227192,0.049463,0.0,0.285762,0.000000,...,0.0,0.089696,0.0,0.000000,0.035702,0.0,0.000000,0.005784,0.150867,WALKING_UPSTAIRS


In [74]:
np.savetxt('uci_merged_dense_test_file2.csv',merged_dense_test_file2, delimiter=',',fmt='%s')
np.savetxt('uci_merged_dense_train_file2.csv',merged_dense_train_file2, delimiter=',',fmt='%s')
np.savetxt('uci_merged_lstm_test_file2.csv',merged_lstm_test_file2, delimiter=',',fmt='%s')
np.savetxt('uci_merged_lstm_train_file2.csv',merged_lstm_train_file2, delimiter=',',fmt='%s')